In [44]:
from moviepy.video.io.VideoFileClip import VideoFileClip

In [ ]:
# Load the video file
video = VideoFileClip("input/PXL_20250222_123802359.mp4")

# Extract the audio
audio = video.audio

# Write the audio to a file
audio.write_audiofile("output/output_audio.mp3")

In [ ]:
import numpy as np
from pydub import AudioSegment

# Load the audio file using pydub
audio_segment = AudioSegment.from_mp3("output/output_audio.mp3")

# Convert audio to numpy array
samples = np.array(audio_segment.get_array_of_samples())

# Calculate the volume in decibels
def calculate_db(samples):
    return 20 * np.log10(np.abs(samples) / np.max(np.abs(samples)))

peak_threshold = -20  # Adjust this threshold as needed

# Calculate decibel values
db_values = calculate_db(samples)
sample_rate = audio_segment.frame_rate
for i, db in enumerate(db_values):
    if db > peak_threshold:
        timestamp = i / sample_rate
        print(f"Peak at sample {i} with {db:.2f} dB")

# Identify peaks
peaks = np.where(db_values > peak_threshold)[0]

# Calculate timestamps for peaks
peak_times = peaks / sample_rate

print(f"Number of peaks: {len(peaks)}")
print(f"Peak positions (in samples): {peaks}")
print(f"Peak times (in seconds): {peak_times}")
